# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import seaborn as sns
sns.set_style("darkgrid")
import sklearn.linear_model

In [80]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables relative to whether someone got admitted or not. Think in terms of for a certain prestige level, how many people got admitted and didnt get admitted

In [81]:
# frequency table for prestige and whether or not someone was admitted
df_freq = pd.crosstab(index=df["prestige"], columns="admit")
print df_freq


col_0     admit
prestige       
1.0          61
2.0         148
3.0         121
4.0          67


In [94]:
df_freq

col_0,admit
prestige,
1.0,61
2.0,148
3.0,121
4.0,67


In [12]:
df.describe()


,admit,gre,gpa,prestige
count,397.000000,397.000000,397.000000,397.000000
mean,0.317380,587.858942,3.392242,2.488665
std,0.466044,115.717787,0.380208,0.947083
min,0.000000,220.000000,2.260000,1.000000
25%,0.000000,520.000000,3.130000,2.000000
50%,0.000000,580.000000,3.400000,2.000000
75%,1.000000,660.000000,3.670000,3.000000
max,1.000000,800.000000,4.000000,4.000000


In [36]:
pd.crosstab(df['admit'], df['prestige'], rownames=['admit']) 

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [13]:
prestige_dum = pd.get_dummies(df['prestige'], prefix='prestige')
print prestige_dum

     prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0               0             0             1             0
1               0             0             1             0
2               1             0             0             0
3               0             0             0             1
4               0             0             0             1
5               0             1             0             0
6               1             0             0             0
7               0             1             0             0
8               0             0             1             0
9               0             1             0             0
10              0             0             0             1
11              1             0             0             0
12              1             0             0             0
13              0             1             0             0
14              1             0             0             0
15              0             0         

#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need one less than the number of classes in the field. For example, prestige includes 4 different values (1.0, 2.0. 3.0, and 4.0). When modeling, we should only use 3 of these values (i.e. 1.0, 2.0, and 3.0). The model will be able to understand when the row actually would be equal to 4.0.


## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [14]:
cols_to_keep = ['admit', 'gre', 'gpa']
df_no_prestige = df[cols_to_keep]

model_data = df_no_prestige.join(prestige_dum[[1, 2, 3]])
print model_data.head()


   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [45]:
def percentage(part, whole):
  return 100 * float(part)/float(whole)

percentage(33,28)

117.85714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [46]:
def percentage(part, whole):
  return 100 * float(part)/float(whole)

percentage(93,243)

38.27160493827161

#### 3.3 Calculate the odds ratio

In [47]:
percentage(33,28)/percentage(93,243)

3.0794930875576036

#### 3.4 Write this finding in a sentenance: 

Answer: one is 3x more likely to be admitted to grad school if they have attended a #1 ranked college 

#### 3.5 Print the cross tab for prestige_4

In [103]:
pd.crosstab(df[df.prestige==4].prestige,df[df.prestige==4].admit)

admit,0,1
prestige,,
4.0,55,12


#### 3.6 Calculate the OR 

In [104]:
percentage(12,55)

21.818181818181817

#### 3.7 Write this finding in a sentence

Answer:

## Part 4. Analysis

In [37]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
df_no_prest = df[cols_to_keep]

model_data = df_no_prest.join(prestige_dum[[1, 2, 3]])
print model_data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [23]:
# manually add the intercept
model_data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [24]:
train_cols = model_data.columns[1:]

#### 4.2 Fit the model

In [25]:
model_data_fit = sm.Logit(model_data['admit'], model_data[train_cols])
complete_data = model_data_fit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [26]:
print complete_data.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 03 May 2017   Pseudo R-squ.:                 0.08166
Time:                        19:02:41   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [27]:
print np.exp(complete_data.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [28]:
ci_int = complete_data.conf_int()
ci_int['OR'] = complete_data.params
ci_int.columns = ['2.5%', '97.5%', 'OR']
print np.exp(ci_int)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: With an OR of 0.5, this means that a student is 50% less likely to be admitted if they went to a school with prestige = 2.


#### 4.6 Interpret the OR of GPA

Answer: For every full increase in GPA, a student is 200% more likely to be admitted. For example, a student with a GPA of 3.0 is twice as likely to be admitted than a student with a GPA of 2.0.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [29]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [38]:

# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(model_data['gre'].min(), model_data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(model_data['gpa'].min(), model_data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
print combos.head()

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]
       0         1    2    3
0  220.0  2.260000  1.0  1.0
1  220.0  2.260000  2.0  1.0
2  220.0  2.260000  3.0  1.0
3  220.0  2.260000  4.0  1.0
4  220.0  2.453333  1.0  1.0


#### 5.1 Recreate the dummy variables

In [39]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
prestige_dummies = pd.get_dummies(combos['prestige'], prefix='prestige')
prestige_dummies.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos_keep = combos[cols_to_keep]
combos_final = combos_keep[cols_to_keep].join(prestige_dummies[[1, 2, 3, 4]])

combos_final['admit_pred'] = result.predict(combos_final[train_cols])
# not sure how to fix this error, I thought I was reproducing what I did from above

IndexError: indices are out-of-bounds

#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.